In [14]:
import csv
import json
import numpy as np
import urllib
from datetime import datetime, timedelta
import time

# writes results of the simulation as a list to a csv 
def write_results(result_list,csv_out_file):

    with open(csv_out_file, 'w', newline='') as f:
        writer = csv.writer(f, delimiter = ' ')
        for elem in result_list:
            writer.writerow(elem)

# pulls the latest data from rki webpage, workaround to pull all entrys not only the first 80 
def pull_rki_data():
    import urllib.request, json 
    from datetime import timedelta, date

    def daterange(start_date, end_date):
        for n in range(int ((end_date - start_date).days)):
            yield start_date + timedelta(n)

    start_date = date(2020, 1, 1)
    end_date = date.today() + timedelta(days=1)
    
    #json_comp =json.dumps({})
    json_comp = []

    #crawls all data from January to March...this might take a while
    for single_date in daterange(start_date, end_date):
        with urllib.request.urlopen("https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0//query?where=Meldedatum%3D%27"+str(single_date.strftime("%Y-%m-%d"))+"%27&objectIds=&time=&resultType=none&outFields=*&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&sqlFormat=none&f=pjson&token=") as url:
            json_data = json.loads(url.read().decode("utf-8"))
            json_comp.append(json_data)# json_comp.append(json_data)
            
    #print("",json_comp)
    return(json_comp)


#loads the json file with the current infected people as well as a csv with population data and nuts3 
# estimates the first date of infection and the number of cases and saves all necessary data into a list
def load_data(csvfile):
    
    res = urllib.request.urlopen('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')
    res_body = res.read()

    data = []
    population_list = []
    landkreis_list= []
      
    with open(csvfile, 'r') as landkreis_file:
            reader = csv.reader(landkreis_file, delimiter=';')
            line = landkreis_file.readline()
            for line in reader:
                population_list.append(line[9:12])
                    


    #arcgis = json.loads(res_body.decode("utf-8"))
    arcgis =  pull_rki_data()
    #Go over all landkreise in arcgis data and extract all different names
    landkreise = []
    
    number = 0
       # print("Element: ",elem)
    for elem in arcgis:
        if 'features' in elem and elem['features']:
            #print("features")
            for feature in elem['features']:
                number = number +1
                if not any(x == feature['attributes']['IdLandkreis'] for x in landkreise):
                    if not feature['attributes']['IdLandkreis'] == '0-1':
                        landkreise.append(feature['attributes']['IdLandkreis'])

    print("",number)
                
    print("Number of landkreise",len(landkreise))
    print("Number of population",len(population_list))
    
    
     
    # for each landkreis estimate the earliest occurence data ---> then get all cases for this day     
    for landkreis in landkreise:

            occurrence_date = 0
            first_occurrence = 0
            first_alert_date = []
            number_of_cases_on_first_date = 0
            number_of_cases = 0
            pop = 0
            nuts3 = ''
            
            for elem in arcgis:
                if 'features' in elem and elem['features']:
                    for feature in elem['features']:
                        total_case_numbers = 0

                        if landkreis == feature['attributes']['IdLandkreis']: 
                            alert_date = feature['attributes']['Meldedatum'] 
                            diff_now_alert_date = 1584835200000 - alert_date #date of today at midnight serves as reference TODO change to date of today dynamically
                            diff_now_alert_date = diff_now_alert_date/(1000*60*60*24)

                            occurrence_date = int(diff_now_alert_date)

                            if occurrence_date > first_occurrence:
                                first_occurrence = occurrence_date
                                first_alert_date =feature['attributes']['Meldedatum'] 

            for elem in arcgis:
                if 'features' in elem and elem['features']:
                    for feature in elem['features']:  
                        if landkreis == feature['attributes']['IdLandkreis']:
                            number_of_cases  = number_of_cases + feature['attributes']['AnzahlFall']
                            if first_alert_date == feature['attributes']['Meldedatum']:
                                number_of_cases_on_first_date = number_of_cases_on_first_date + feature['attributes']['AnzahlFall']


            for line in population_list:
                if not line[0] == '0-1':
                    if int(line[0]) == int(landkreis):
                        pop = int(line[1])
                        nuts3 = line[2]

            landkreis_list.append([int(landkreis),number_of_cases_on_first_date,number_of_cases,first_occurrence, pop, nuts3])
            
    return landkreis_list
            
csvfile = 'C:/Users/thomass/Desktop/RKI-COVID-mit-Einwohnerzahl_und_NUTS3_pro_Fall.csv'
landkreis_list = load_data(csvfile)

write_results(landkreis_list,'C:/Users/thomass/Desktop/RKI-COVID_out.csv')

print("",landkreis_list)
# list with: 
# Landkreis ID
# number_of_cases_on_first_date
# number_of_cases_now
# first_occurrence in days
# population
# nuts3






 8919
Number of landkreise 405
Number of population 8239
 [[9181, 1, 18, 54, 120071, 'DE21E'], [9188, 1, 83, 54, 136092, 'DE21L'], [9162, 1, 584, 53, 1471508, 'DE212'], [9179, 1, 46, 53, 219320, 'DE21C'], [9189, 2, 44, 51, 177089, 'DE21M'], [3401, 1, 6, 47, 77607, 'DE941'], [9184, 1, 220, 47, 348871, 'DE21H'], [9178, 1, 212, 39, 179116, 'DE21B'], [8117, 1, 110, 26, 257253, 'DE114'], [8416, 1, 114, 26, 227331, 'DE142'], [5370, 3, 808, 25, 254322, 'DEA29'], [8116, 1, 322, 25, 533859, 'DE113'], [8311, 1, 149, 25, 230241, 'DE131'], [8315, 1, 133, 25, 262795, 'DE132'], [8325, 1, 10, 25, 139455, 'DE135'], [5166, 1, 39, 24, 298935, 'DEA1E'], [8115, 1, 67, 24, 391640, 'DE112'], [8118, 1, 176, 24, 543984, 'DE115'], [9562, 1, 17, 24, 111962, 'DE252'], [1060, 1, 29, 23, 276032, 'DEF0D'], [5170, 1, 66, 23, 459809, 'DEA1F'], [5314, 1, 73, 23, 327258, 'DEA22'], [5334, 1, 178, 23, 555465, 'DEA2D'], [5962, 1, 31, 23, 412120, 'DEA58'], [6531, 1, 56, 23, 268876, 'DE721'], [7312, 1, 14, 23, 99845, 'DEB32